In [1]:
!pip --version
!pip install pyspark py4j

pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [40]:
from pyspark.sql import SparkSession

In [41]:
spark = SparkSession.builder.appName('MyAppSpark') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

In [42]:
# Чтение данных из CSV файлов
df_actors = spark.read.csv('/content/sample_data/actors.csv', header=True, inferSchema=True)
df_movies = spark.read.csv('/content/sample_data/movies.csv', header=True, inferSchema=True)
df_movies_actors = spark.read.csv('/content/sample_data/movie_actors.csv', header=True, inferSchema=True)

In [43]:
# Регистрация DataFrame как временные таблицы
df_actors.createOrReplaceTempView('actors')
df_movies.createOrReplaceTempView('movies')
df_movies_actors.createOrReplaceTempView('movies_actors')

In [61]:
spark.sql('''
          SELECT *
          FROM actors
          LIMIT 2
          ''').show()

spark.sql('''
          SELECT *
          FROM movies
          LIMIT 2
          ''').show()

spark.sql('''
          SELECT *
          FROM movies_actors
          LIMIT 2
          ''').show()

+--------+-------+----------+-------+
|actor_id|   name|birth_date|country|
+--------+-------+----------+-------+
|       1|Actor_1|1960-12-31| Canada|
|       2|Actor_2|1962-12-31|     UK|
+--------+-------+----------+-------+

+--------+-------+------+------------+-------------+
|movie_id|  title| genre|release_date|       budget|
+--------+-------+------+------------+-------------+
|       1|Movie_1|Horror|  2000-12-31|8.660058311E7|
|       2|Movie_2|Comedy|  2001-12-31|1.274740083E7|
+--------+-------+------+------------+-------------+

+--------+--------+
|movie_id|actor_id|
+--------+--------+
|       1|      25|
|      16|       5|
+--------+--------+



In [44]:
#топ-5 жанров по количеству фильмов.
df_top5_genre = spark.sql("""
          SELECT
            genre,
            count(movie_id) as num_movies
          FROM movies
          GROUP BY genre
          ORDER BY num_movies desc
          LIMIT 5
          """)
df_top5_genre.show()

+------+----------+
| genre|num_movies|
+------+----------+
| Drama|         6|
|Action|         6|
|Comedy|         4|
|Horror|         2|
|Sci-Fi|         2|
+------+----------+



In [58]:
# актер с наибольшим количеством фильмов.

df_top_actor = spark.sql("""
    SELECT
        a.name,
        count(ma.movie_id) num_movies
    FROM actors a
    JOIN movies_actors ma USING(actor_id)
    GROUP BY a.actor_id, a.name
    ORDER BY num_movies desc
    LIMIT 1
""")

df_top_actor.show()

+--------+----------+
|    name|num_movies|
+--------+----------+
|Actor_17|         5|
+--------+----------+



In [60]:
# средний бюджет фильмов по жанрам.

query = """
        SELECT
            genre,
            avg(budget) as avg_budget
        FROM
            movies
        GROUP BY genre
        """
genre_avg_budget = spark.sql(query)
genre_avg_budget.show()

+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
| Drama| 6.076021856166667E7|
|Horror|      8.7281876775E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
|Sci-Fi|       7.809715175E7|
+------+--------------------+



In [64]:
# Найдите фильмы, в которых снялось более одного актера из одной страны.
query = """
        SELECT
            m.title,
            a.country,
            count(a.actor_id)as num_actors
        FROM
            movies_actors ma
        JOIN movies m USING(movie_id)
        JOIN actors a USING(actor_id)
        GROUP BY m.title, a.country
        HAVING num_actors > 1
        """
movies_with_multiple_actors_same_country = spark.sql(query)

movies_with_multiple_actors_same_country.show()

+--------+---------+----------+
|   title|  country|num_actors|
+--------+---------+----------+
| Movie_7|    India|         2|
| Movie_3|      USA|         2|
|Movie_10|       UK|         2|
|Movie_15|    India|         2|
|Movie_18|Australia|         2|
| Movie_1|    India|         3|
| Movie_2|      USA|         2|
| Movie_7|      USA|         2|
|Movie_10|      USA|         2|
+--------+---------+----------+

